<a href="https://colab.research.google.com/github/butturinipedro-eng/estabilidade/blob/main/Trabalho2_estabilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.integrate import quad
import math
np.printoptions(precision=4, linewidth = 80, suppression = True)
import pandas as pd


#Trabalho 2

#Ideia Geral

* Fazer Ybarra, a partir disso e da matriz de tensões calcular as correntes injetadas e as potências injetadas em cada barra.

* Com as potencias em cada barra e a potencia de cada gerador consigo calcular o numero de geradores

* Com isso calculo a tensão transitória interna dos geradores já que tenho o Xd e N




#Letra a) Numero de maquinas por usina

In [98]:
#Funçao para modificar linhas e colunas da matriz
def add_brach(Y, de, para, y):
  Y[de, para] -= y
  Y[para, de] -= y
  Y[de, de] += y
  Y[para, para] += y



In [99]:
#Jeito do professor
YLT = np.zeros((4,4), dtype = complex)

#LT 1-2 (de = 0 , para = 1)
add_brach(YLT, de = 0, para = 1, y=y12)

#LT 1-3 (de = 0 , para = 2)
add_brach(YLT, de = 0, para = 2, y=y13)

#LT 2-4 (de = 1 , para = 3)
add_brach(YLT, de = 1, para = 3, y=y24)

#LT 3-4 (de = 2 , para = 3)
add_brach(YLT, de = 2, para = 3, y=y34)
display(YLT)


array([[0.-15.j, 0.+10.j, 0. +5.j, 0. +0.j],
       [0.+10.j, 0.-20.j, 0. +0.j, 0.+10.j],
       [0. +5.j, 0. +0.j, 0.-15.j, 0.+10.j],
       [0. +0.j, 0.+10.j, 0.+10.j, 0.-20.j]])

In [100]:
#parametros
#Sistema
base_g1 = 250
base_g2 = 200
base_sis = 100
wb = 2*np.pi*60
Sc3 = complex(300, 100)/base_sis
Sc4 = complex(100, 40)/base_sis
Qc3 = -100/base_sis
Qr4 = 50/base_sis

Yrede = np.zeros((4,4))
y12 = 1/(0.1*1j)
y13 = 1/(0.2*1j)
y24 = 1/(0.1*1j)
y34 = 1/(0.1*1j)
Ylt = np.zeros((4,4), dtype=complex)

Ylt[0,0] += y12 + y13
Ylt[1,1] += y24 + y34 # Assuming y12 is connected to bus 1, and y34 to bus 2
Ylt[2,2] += y34 + y13 # Assuming y13 is connected to bus 2, and y34 to bus 3
Ylt[3,3] += y24 + y34

Ylt[0,1] = Ylt[1,0] = -y12 # Connection between bus 0 and bus 1
Ylt[0,2] = Ylt[2,0] = -y13 # Connection between bus 0 and bus 2
Ylt[1,3] = Ylt[3,1] = -y24 # Connection between bus 1 and bus 3
Ylt[2,3] = Ylt[3,2] = -y34 # Connection between bus 2 and bus 3

#Jeito do professor
Y = np.zeros((4,4), dtype = complex)

#LT 1-2 (de = 0 , para = 1)
add_brach(Y, de = 0, para = 1, y=y12)

#LT 1-3 (de = 0 , para = 2)
add_brach(Y, de = 0, para = 2, y=y13)

#LT 2-4 (de = 1 , para = 3)
add_brach(Y, de = 1, para = 3, y=y24)

#LT 3-4 (de = 2 , para = 3)
add_brach(Y, de = 2, para = 3, y=y34)
display(Y)

print("\nMatriz Ybarra: \n", Ylt, "\n\n")

#Tensões nas barras
V = np.zeros((4,1), dtype = complex)
V[0, 0] =  1.0500 * np.exp(1j * np.deg2rad(0.00))
V[1, 0] = 1.0100 * np.exp(1j * np.deg2rad(-3.11))
V[2, 0] = 0.8887 * np.exp(1j * np.deg2rad(-24.36))
V[3, 0] = 0.8867 * np.exp(1j * np.deg2rad(-16.51))
print("Matriz das tensões : \n", V, "\n\n")
#Note que P3 = -3pu (carga de 300MW) e P4 = -1pu (Carga de 100 MW)

#Matriz das correntes injetadas
I_inj = Ylt @ V
print("Correntes injetadas (pu): \n", I_inj, "\n\n")

#Pot ativas e reativas nos geradores
S = V*np.conj(I_inj)
print("Matriz S \n" ,S, "\n\n")
mag_S1 = np.abs(S[0]) #valor real de pot (MW)
mag_S2 = np.abs(S[1]) #valor real de pot (MW)
print("Potencia injetada na barra 1 =" ,S[0] , " pu\nMagnitude da potência instalada na barra 1 = ", mag_S1, "pu \n\n")
print("Potencia injetada na barra 2 =" ,S[1] , "pu \nMagnitude da potência instalada na barra 2 = ", mag_S2, "pu \n\n")

#Número de geradores
base_g1 = 250
base_g2 = 200
Pg1 = mag_S1*base_sis
N1 = math.ceil(Pg1/base_g1)
print("Numero de geradores N1 = ", N1," \n\n")
Pg2 = mag_S2*base_sis
N2 = math.ceil(Pg2/base_g2)
print("Numero de geradores N2 = ", N2," \n\n")

array([[0.-15.j, 0.+10.j, 0. +5.j, 0. +0.j],
       [0.+10.j, 0.-20.j, 0. +0.j, 0.+10.j],
       [0. +5.j, 0. +0.j, 0.-15.j, 0.+10.j],
       [0. +0.j, 0.+10.j, 0.+10.j, 0.-20.j]])


Matriz Ybarra: 
 [[ 0.-15.j -0.+10.j -0. +5.j  0. +0.j]
 [-0.+10.j  0.-20.j  0. +0.j -0.+10.j]
 [-0. +5.j  0. +0.j  0.-15.j -0.+10.j]
 [ 0. +0.j -0.+10.j -0.+10.j  0.-20.j]] 


Matriz das tensões : 
 [[1.05      +0.j        ]
 [1.00851249-0.05479562j]
 [0.80958067-0.3665608j ]
 [0.85014149-0.25198479j]] 


Correntes injetadas (pu): 
 [[ 2.38076022-1.61697174j]
 [ 1.42393547-1.16883485j]
 [-2.97856416+1.60770481j]
 [-0.82613153+1.17810179j]] 


Matriz S 
 [[ 2.49979823+1.69782033j]
 [ 1.50010373+1.10075912j]
 [-3.00070954-0.20974188j]
 [-0.99919242-0.79338063j]] 


Potencia injetada na barra 1 = [2.49979823+1.69782033j]  pu
Magnitude da potência instalada na barra 1 =  [3.02185126] pu 


Potencia injetada na barra 2 = [1.50010373+1.10075912j] pu 
Magnitude da potência instalada na barra 2 =  [1.86064017] pu 


Numero de geradores N1 =  2  


Numero de geradores N2 =  1  




/tmp/ipython-input-3583617743.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  N1 = math.ceil(Pg1/base_g1)
/tmp/ipython-input-3583617743.py:75: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  N2 = math.ceil(Pg2/base_g2)


#Letra b) Tensão trasitória interna dos geradores
E' = Vt + j* N * X'd * I_inj

In [101]:
#Passando X'd dos geradores para a base do sistema
X1 = 0.2
Xd1 = (1/N1)*X1*base_sis/base_g1
print("X'd1 = ", Xd1, "\n")
H1 = 5

X2 = 0.3
Xd2 = (1/N2)*X2*base_sis/base_g2
print("X'd2 = ", Xd2, "\n")
H2 = 4

E_linha1_ph = V[0] + 1j * Xd1 * I_inj[0]
E_linha2_ph = V[1] + 1j * Xd2 * I_inj[1]

E1 = abs(E_linha1_ph)
d1 = np.angle(E_linha1_ph)
E2 = abs(E_linha2_ph)
d2 = np.angle(E_linha2_ph)

print("E'1 = ", E1, "|", d1*180/np.pi, "° \n")
print("E'2 = ", E2, "|", d2*180/np.pi, "° \n")

X'd1 =  0.04 

X'd2 =  0.15 

E'1 =  [1.11873939] | [4.88309562] ° 

E'2 =  [1.19444024] | [7.63979824] ° 



# Tratamento das cargas e elementos em derivação

Compensação em derivação

In [102]:
yc3 = -Qc3*1j
yr4 = -Qr4*1j
print("yc3 = ", yc3, "\n")
print("yr4 = ", yr4, "\n")

yc3 =  1j 

yr4 =  (-0-0.5j) 



Cargas com impedancia constante

In [103]:
#S*/V^2
yL4 = Sc4.conjugate()/abs(V[3])**2
yL3 = Sc3.conjugate()/abs(V[2])**2
print("yL3 = ", yL3, "\n")
print("yL4 = ", yL4, "\n")

yL3 =  [3.79848919-1.26616306j] 

yL4 =  [1.2718813-0.50875252j] 



## Matrizes de sistema

In [111]:
Ybb = YLT.copy()
Ybg = np.zeros((4,2), dtype=complex) #4 barras e 2 geradores, tem numero de elementos diferentes de zero = numero de geradores
Ygg = np.zeros((2,2), dtype=complex) #2 geradores

#banco de capacitores na barra 3
Ybb[2,2] += yc3

#banco de reatores na barra 4
Ybb[3,3] += yr4

#Carga na barra 3
Ybb[2,2] += yL3

#Carga na barra 4
Ybb[3,3] += yL4


#Gerador 1 na barra 1
yg1 = 1/(1j*Xd1)
Ybb[0,0] += yg1
Ybg[0,0] -= yg1
Ygg[0,0] += yg1

#Gerador 2 na barra 2
yg2 = 1/(1j*Xd2)
Ybb[1,1] += yg2
Ybg[1,1] -= yg2
Ygg[1,1] += yg2
print("\nMatriz Ybb: \n")
display(pd.DataFrame(Ybb))
print("\nMatriz Ybg: \n")
display(pd.DataFrame(Ybg))
print("\nMatriz Ygg: \n")
display(pd.DataFrame(Ygg))


Matriz Ybb: 



/tmp/ipython-input-4126429903.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Ybb[2,2] += yL3
/tmp/ipython-input-4126429903.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Ybb[3,3] += yL4


,0,1,2,3
0,0.0-40.0j,0.000000+10.000000j,0.000000+ 5.000000j,0.000000+ 0.000000j
1,0.0+10.0j,0.000000-26.666667j,0.000000+ 0.000000j,0.000000+10.000000j
2,0.0+ 5.0j,0.000000+ 0.000000j,3.798489-15.266163j,0.000000+10.000000j
3,0.0+ 0.0j,0.000000+10.000000j,0.000000+10.000000j,1.271881-21.008753j



Matriz Ybg: 



,0,1
0,0.0+25.0j,0.000000+0.000000j
1,0.0+ 0.0j,0.000000+6.666667j
2,0.0+ 0.0j,0.000000+0.000000j
3,0.0+ 0.0j,0.000000+0.000000j



Matriz Ygg: 



,0,1
0,0.0-25.0j,0.000000+0.000000j
1,0.0+ 0.0j,0.000000-6.666667j


# Matriz reduzida pré falta e potência pré falta
Yred = Ygg - Ygb(Ybb)^-1Ybg


X = (Ybb)^-1 Ybg


Ybb*x = Ybg

In [121]:
X = np.linalg.solve(Ybb, Ybg)
Yred_pre = Ygg - Ybg.T @ X
print("\n Yred_pre: \n")

Gred_pre = np.real(Yred_pre)
Bred_pre = np.imag(Yred_pre)

display(pd.DataFrame(Yred_pre))


 Yred_pre: 



,0,1
0,1.460820-4.961240j,0.652309+3.100526j
1,0.652309+3.100526j,0.301585-3.999650j


Pk = SUM( Ek * Em (Gkm * cos(dkm)+ Bkm * (sen(dkm))


P1 =



In [129]:
d12 = d1 - d2
def Pe1_pre(d12):
  return Gred_pre[0,0]*E1**2 + E1*E2*(Gred_pre[0,1]*np.cos(d12) + Bred_pre[0,1]*np.sin(d12))

def Pe2_pre(d12):
  return Gred_pre[1,1]*E2**2 + E1*E2*(Gred_pre[0,1]*np.cos(d12) - Bred_pre[0,1]*np.sin(d12))

print("P1_pre = ", Pe1_pre(d12), "\n\n")
print("P2_pre = ", Pe2_pre(d12), "\n\n")

P1_pre =  [2.49971722] 


P2_pre =  [1.50018278] 




#Matriz reduzida e Potência durante falta


In [139]:
#Falta na barra 2
Xf = 1e-6
yf = 1/(1j*Xf)
Ybb_dur = Ybb.copy()
Ybb_dur[1,1] += yf

X = np.linalg.solve(Ybb_dur, Ybg)
Yred_dur = Ygg - Ybg.T @ X
print("\n Yred_dur: \n")

Gred_dur = np.real(Yred_dur)
Bred_dur = np.imag(Yred_dur)

display(pd.DataFrame(Yred_dur))

def Pe1_dur(d12):
  return Gred_dur[0,0]*E1**2 + E1*E2*(Gred_dur[0,1]*np.cos(d12) + Bred_dur[0,1]*np.sin(d12))

def Pe2_dur(d12):
  return Gred_dur[1,1]*E2**2 + E1*E2*(Gred_dur[0,1]*np.cos(d12) - Bred_dur[0,1]*np.sin(d12))

print("P1_dur = ", Pe1_dur(d12), "\n\n")
print("P2_dur = ", Pe2_dur(d12), "\n\n")


 Yred_dur: 



,0,1
0,0.347939-8.531980j,0.000005+0.000052j
1,0.000005+0.000052j,0.000000-6.666622j


P1_dur =  [0.43547539] 


P2_dur =  [9.98267595e-06] 


